# WEB SCRAPING BOCG

#### Instalamos la librerias necesarias si no las tenemos instaladas previamente, para ello descomentamos borrando "#".

In [1]:
#install.packages("pdftools")
#install.packages("tesseract")
#install.packages("stringr")

#### Seleccionamos las librerias que vamos a utilizar.

In [3]:
library(pdftools)
library(tesseract)
library(stringr)

## Extracción de los BOCG

### Criterios de búsqueda

En este punto colocamos los criterios de búsqueda para cada BOCG y actas que contenga.

In [4]:
criterio1 <- "Juan Carlos"
criterio2 <- "Monarquía"
criterio3 <- "emérito"

Creamos un vector con el **Número de BOGC's en cada legislatura**. En este caso hemos colocado a cero las 13 primeras legislaturas para simplificar la búsqueda y en la legislatura 14 terminamos en el bocg nº 236.

In [5]:
n_bocgs <- c(0,0,0,0,0,0,0,0,0,0,0,0,0,236)

Descargamos las páginas de los BOGC para las legislaturas seleccionadas anteriormente, con los criterios de búsqueda.

### Inicialización de parámetros

Ponemos a cero las variables donde vamos a almacenar los resultados.

In [6]:
bocg <- list() # Lista que almacena los BOCG con los criterios de búsqueda
i = 0 # contador

### Iteración por los BOGS's de cada legislatura

Recorremos los BOGS's de cada una de las legislaturas y nos quedamos con las páginas de aquellos que contienen alguno de los criterios de búsqueda.

In [7]:
# Para cada legislatura y BOGC publicado     
for (leg in 14:14){ # A modo de ejemplo solo vamos a recorrer la legislatura 14
    for (n_bocg in 234:n_bocgs[leg]){ # Solo descargamos del bocg 234 al último, 236 en este caso.
        
        # url del Boletín oficial
        url_bocg <- paste0("https://www.congreso.es/public_oficiales/L",leg,"/CONG/BOCG/D/BOCG-",leg,"-D-",n_bocg,".PDF")
        
        # Descargamos el boletín
        download.file(url_bocg, "BOCG.pdf", mode = "wb")
        
        # Extraemos el author, version, etc
        info <- pdf_info("BOCG.pdf")
        
        # Extraemos el texto del boletín
        txt <- pdf_text("BOCG.pdf")
        
        # Comprobamos si existen páginas en el pdf con los criterios de búsqueda y si hay actas
        acta_true <- stringr::str_subset(txt, "Ver_texto") # Los pdf con actas contienen el texto "Ver_texto"
        resul1 <- stringr::str_subset(txt, criterio1)
        resul2 <- stringr::str_subset(txt, criterio2)
        resul3 <- stringr::str_subset(txt, criterio3)

        # Si el pdf contiene actas
        if (length(acta_true) > 0){ 
          # Si existen páginas con los criterios búsqueda las almacenamos en la lista "bocg"
          if (length(resul1) | length(resul2) | length(resul3) > 0){
            i <- i + 1
            bocg_info <-info[["keys"]][["Subject"]]
            
            # Imprimimos la información de los BOGC's que va descargando
            print(paste0("BOGC: ",bocg_info))

            # Almaceno las páginas para cada pdf en la lista "bocg"
            bocg[[i]] <-list(c(leg, n_bocg, bocg_info, resul1, resul2, resul3))

            # Cada elemento de la lista "bocg" lo guardamos en un archivo txt
            write.table(x = bocg[[i]][[1]], file = paste0("./ouput/",bocg[[i]][[1]][3],".txt"))
          }
        }
    }
}

[1] "BOGC: Número 235 de 8 de marzo de 2021"


A modo de ejemplo, mostramos las páginas que contienen los criterios de búsqueda para uno de los BOCG descargados.

In [8]:
print(bocg[1][1])

[[1]]
[[1]][[1]]
 [1] "14"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

## Extracción de las actas (para un BOCG)

A modo de ejemplo vamos a realizar la descarga del texto de las actas para el BOCG nº 235 de la legislatura 14. Más abajo se presenta el código para la descarga completa para todas las actas de los BOCG's con los criterios de busqueda.

### Iteración por las actas del BOCG seleccionado

Recorremos las actas de cada una de las páginas del BOCG seleccionado y nos quedamos con las actas que contienen alguno de los criterios de búsqueda.

In [22]:
num_reg <- list() # Lista que almacena el número de registro del acta contenida en el BOCG
txt_acta <- list() # Lista que almacena el texto del acta

### Iteración por las actas del BOCG seleccionado

Recorremos las actas de cada una de las páginas del BOCG seleccionado y nos quedamos con las actas que contienen alguno de los criterios de búsqueda.

In [25]:
# Número de registro de cada una de las actas contenidas en el BOCG
l <- length(bocg[[1]][[1]]) #corresponde con el BOCG de la lista anterior
for (k in 4:l){
    reg <- str_extract_all(bocg[[1]][[1]][k],"reg.......")
    num_loc <- str_locate_all(reg, "[0-9]+")[[1]]
    num_reg <- str_sub(reg, num_loc[, "start"], num_loc[, "end"])

    # Para cada acta incluida en el BOCG
    for (j in num_reg){
        # Mostramos los números de registro para cada BOCG
        print(paste0("BOCG: ",bocg[[1]][[1]][3]," Números de registro del acta: ",j))
        num <- str_pad(j, 7, pad = "0")

        # url del acta
        #for (h in 0:10){
        acta <- paste0("https://www.congreso.es/entradap/l",bocg[[1]][[1]][1],"p/e",9,"/e_",num,"_n_000.pdf")
        pdf <- 200 # Longitud mínima del texto del acta

        # Descargamos el acta
        try(pdf <- download.file(acta, "acta.pdf", mode = "wb"),silent=TRUE)

        if (pdf == 0){
          # Extraemos el author, version, etc
          info_acta <- pdf_info("acta.pdf")

          # Extraemos el texto del acta
          txt_acta <- pdf_text("acta.pdf")


          ##################################################################################
          # Separamos las actas con imagenes de las que son texto y las guardamos en un txt
          ##################################################################################

          # Si las actas son imágenes, extraemos el texto de la imagen 
          if(str_length(txt_acta) < 200){
            pdf_ocr <- pdf_ocr_text("acta.pdf")

            # Comprobamos si existen actas con los criterios de búsqueda
            resul_acta1 <- stringr::str_subset(pdf_ocr, criterio1)
            resul_acta2 <- stringr::str_subset(pdf_ocr, criterio2)
            resul_acta3 <- stringr::str_subset(pdf_ocr, criterio3)

            # Si existen actas con los criterios búsqueda Guardamos el texto en un txt
            if (length(resul_acta1) | length(resul_acta2) | length(resul_acta3) > 0){
                # Guardamos el texto en un txt
                write.table(x = pdf_ocr, file = paste0("./ouput/acta",j,".txt"), )
            }
            # Si las actas son texto, extraemos el texto
          } 
          else{
            # Comprobamos si existen actas con los criterios de búsqueda
            resul_acta1 <- stringr::str_subset(txt_acta, criterio1)
            resul_acta2 <- stringr::str_subset(txt_acta, criterio2)
            resul_acta3 <- stringr::str_subset(txt_acta, criterio3)


            # Si existen actas con los criterios búsqueda Guardamos el texto en un txt
            if (length(resul_acta1) | length(resul_acta2) | length(resul_acta3) > 0){
                write.table(x = pdf_ocr[[i]], file = paste0("./ouput/acta",j,".txt"), )
            }
          } 
          ##################################################################################
        }
      }
}

Warning message in stri_locate_all_regex(string, pattern, omit_no_match = TRUE, :
"argument is not an atomic vector; coercing"Warning message in stri_sub(string, from = start, to = end):
"argument is not an atomic vector; coercing"

[1] "BOCG: Número 235 de 8 de marzo de 2021 Números de registro del acta: 94536"


Warning message in if (str_length(txt_acta) < 200) {:
"la condición tiene longitud > 1 y sólo el primer elemento será usado"

Converting page 1 to acta_1.png...

PDF error: No display font for 'ArialUnicode'


...

 done!
[1] "BOCG: Número 235 de 8 de marzo de 2021 Números de registro del acta: 91247"
Converting page 1 to acta_1.png...

PDF error: No display font for 'ArialUnicode'


 done!
[1] "BOCG: Número 235 de 8 de marzo de 2021 Números de registro del acta: 91248"
Converting page 1 to acta_1.png...

PDF error: No display font for 'ArialUnicode'


 done!


In [ ]:
bocg[[1]]

## Extracción de las actas

### Inicialización de parámetros

Ponemos a cero las variables donde vamos a almacenar los resultados.

In [ ]:
num_reg <- list() # Lista que almacena el número de registro del acta contenida en el BOCG
txt_acta <- list() # Lista que almacena el texto del acta

### Iteración por las actas del BOCG seleccionado

Recorremos las actas de cada una de las páginas del BOCG seleccionado y nos quedamos con las actas que contienen alguno de los criterios de búsqueda.

In [ ]:
# Para cada BOCG en las lista "bocg"
for (m in 1:length(bocg)){
  
  # Número de registro de cada una de las actas contenidas en el BOCG
  l <- length(bocg[[m]][[1]])
  for (k in 4:l){
    reg <- str_extract_all(bocg[[m]][[1]][k],"reg.......")
    
    num_loc <- str_locate_all(reg, "[0-9]+")[[1]]
    num_reg <- str_sub(reg, num_loc[, "start"], num_loc[, "end"])
    
    # Para cada acta incluida en el BOCG
    for (j in num_reg){
      # Mostramos los números de registro para cada BOCG
      print(paste0("BOCG: ",bocg[[m]][[1]][3]," Números de registro del acta: ",j))
      num <- str_pad(j, 7, pad = "0")
      
      # url del acta
      for (h in 0:10){
        acta <- paste0("https://www.congreso.es/entradap/l",bocg[[m]][[1]][1],"p/e",h,"/e_",num,"_n_000.pdf")
        pdf <- 200
        
        # Descargamos el acta
        try(pdf <- download.file(acta, "acta.pdf", mode = "wb"),silent=TRUE)
        
        if (pdf == 0){
          # Extraemos el author, version, etc
          info_acta <- pdf_info("acta.pdf")
          
          # Extraemos el texto del acta
          txt_acta <- pdf_text("acta.pdf")
          
          
          # Si las actas son imágenes, extraemos el texto de la imagen 
          if(str_length(txt_acta) < 200){
            pdf_ocr <- pdf_ocr_text("acta.pdf")
            
            # Comprobamos si existen actas con los criterios de búsqueda
            resul_acta1 <- stringr::str_subset(pdf_ocr, criterio1)
            resul_acta2 <- stringr::str_subset(pdf_ocr, criterio2)
            resul_acta3 <- stringr::str_subset(pdf_ocr, criterio3)
            
            # Si existen actas con los criterios búsqueda Guardamos el texto en un txt
            if (length(resul_acta1) | length(resul_acta2) | length(resul_acta3) > 0){
              # Guardamos el texto en un txt
              write.table(x = pdf_ocr, file = paste0("./ouput/acta",j,".txt"), )
            }
            # Si las actas son texto, extraemos el texto
          } else{
            # Comprobamos si existen actas con los criterios de búsqueda
            resul_acta1 <- stringr::str_subset(txt_acta, criterio1)
            resul_acta2 <- stringr::str_subset(txt_acta, criterio2)
            resul_acta3 <- stringr::str_subset(txt_acta, criterio3)
            
            # Si existen actas con los criterios búsqueda Guardamos el texto en un txt
            if (length(resul_acta1) | length(resul_acta2) | length(resul_acta3) > 0){
              write.table(x = pdf_ocr[[i]], file = paste0("./ouput/acta",j,".txt"), )
            }
          } 
        }
      }  
    }
  }
}